In [1]:
import serial
from serial.tools import list_ports
from time import sleep


In [ ]:
class communicator():
    
    def __init__(self, verbose: bool=True):
        self.port = self.find_device(verbose)
        self.s = serial.Serial(self.port,115200,timeout=5)# establish connection with the device and open the device
        self.verbose = verbose
        self.com_setup()
        self.com_delay = 0
        self.rx_delay = 0.1
        
    def find_device(self, verbose: bool) -> str:
        ports = list_ports.comports()
        #For Debugging
        #print("Available Serial Ports:")
        #for p in ports:
            #print(f"Port:{p.device} | Description:{p.description} | HWID:{p.hwid}")
            
        possible_devices = list()
        i=0
        
        if verbose: # for optional debugging
            print("Available COMs:")
            
        for p in ports:
            if "CP210x" in p.description: # Checking a device with CP210x in description is connected to the com_port
                possible_devices.append(p.device)
                if verbose:
                    print("%i >" %i, end='')
                i+=1
                
            if verbose: print("\t%-5s: %-30s [%s]" %(p.device, p.description, p.hwid))
        # For returning the device name     
        if not verbose and len(possible_devices) !=1:
            return None
        elif len(possible_devices) ==0:
            print("NO USABLE DEVICE FOUND")
            return None
        elif len(possible_devices)>1:
            print("Type Nr. to Use")
            return possible_devices[int(input())]
        else:
            return possible_devices[0]
        
    def com_setup(self):
        print_str = str()
        
        self.s.reset_input_buffer() # clears all the data waiting in the input buffer
        self.s.reset_output_buffer() #clears all the data waiting in the output buffer
        
        self.s.write(b"restart\n") #RESTARTS THE dEVICE THIS IS an ESP 32 feature: ESP.restart()
        sleep(1)
        
        print_str += self.s.read_all().decode(errors="ignore") # read_all() Reads all available bytes currently waiting in the serial input buffer.
                                                # .decode() Converts the byte read from the serial port into a string
        print(print_str)#
        self.s.write(b"print_mode -m RAW\n")
        sleep(0.5)
        print_str += self.s.read_all().decode(errors='ignore')
        
        if self.verbose:
            print(print_str)
            
    
    def command (self,command_string:str, read:bool=False) -> int:
        self.s.reset_input_buffer()
        txt_str = command_string + "\n" # Appends \n to command_string
    
    
        sleep(self.com_delay)# setup a delay , 0 sec initially
    
        self.s.write(txt_str.encode()) #txt_str.encode() -> encodes the string bytes(because UTF-8 by default) and self.s.write send it to the device
    
        if read is False:
            return None
    
        sleep(self.rx_delay)
    
        self.s.read_until(b"r[") # return type bytes, Read bytes from serial port until it encounters the string "r[", thr device sends data as r[]. This line makes sure that we have reached beginning of a valid response frame
        raw_data_str = self.s.read_until(b"]").decode() # Continues reading until a closing "]" byte is found, then decodes the captured bytes (from after "r[" up to and including "]") into a string.
        if self.verbose():
            print(raw_data_str)
        
        
        
        data, length = raw_data_str.split("#") # breaks the string into two substrings data and length whenever # appears.
        data_mask = (1<<int(length,16))-1 # int(length,16)-> coverts the length from a hexadecimal value to a decimal integer, then creates a bit mask like ob11111111
        data = int(data,16) # coverts data from hex to integer
    
        return data,data_mask
    
    

        
    
        

In [3]:
comm = communicator() # instance for running the communicator()

Available COMs:
0 >	COM4 : Silicon Labs CP210x USB to UART Bridge (COM4) [USB VID:PID=10C4:EA60 SER=0001 LOCATION=1-3]
ets Jul 29 2019 12:21:46

rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid headets Jul 29 2019 12:21:46

rst:0x10 (RTCWDT_RTC_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid heaets Jul 29 2019 12:21:46

rst:0x10 (RTCWDT_RTC_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff
invalid header: 0xffffffff

ets Jul 29 2019 12:21:46

rst:0x1 (POWERON_RESET),boot:0x13 (SP

In [ ]:

#Decodes data from the device. Extracts a hexadecimal data value and bit length , converts these into integers, apples bitmask to the data, and returns both




def command (self,command_string:str, read:bool=False) -> int:
    self.s.reset_input_buffer()
    txt_str = command_string + "\n" # Appends \n to command_string
    
    
    sleep(self.com_delay)# setup a delay , 0 sec initially
    
    self.s.write(txt_str.encode()) #txt_str.encode() -> encodes the string bytes(because UTF-8 by default) and self.s.write send it to the device
    
    if read is False:
        return None
    
    sleep(self.rx_delay)
    
    self.s.read_until(b"r[") # return type bytes, Read bytes from serial port until it encounters the string "r[", thr device sends data as r[]. This line makes sure that we have reached beginning of a valid response frame
    raw_data_str = self.s.read_until(b"]").decode() # Continues reading until a closing "]" byte is found, then decodes the captured bytes (from after "r[" up to and including "]") into a string.
    if self.verbose():
        print(raw_data_str)
        
        
        
    data, length = raw_data_str.split("#") # breaks the string into two substrings data and length whenever # appears.
    data_mask = (1<<int(length,16))-1 # int(length,16)-> coverts the length from a hexadecimal value to a decimal integer, then creates a bit mask like ob11111111
    data = int(data,16) # coverts data from hex to integer
    
    return data,data_mask
    
    
    